In [1]:
import pandas as pd
import random 
import numpy as np
from scipy.spatial import distance_matrix
import time
import warnings
from tqdm import tqdm 
warnings.filterwarnings('ignore')

In [2]:
tsp_c = pd.read_csv("TSPC.csv", sep=';', header=None)
tsp_d = pd.read_csv("TSPD.csv", sep=';', header=None)

In [3]:
tsp_c

,0,1,2
0,1510,353,42
1,3042,123,241
2,2145,389,731
3,1030,365,993
4,2125,360,72
...,...,...,...
195,1023,1258,98
196,3192,1271,591
197,3735,1879,730
198,3983,299,654


In [4]:
data = tsp_c.sample(frac=1)
data

,0,1,2
191,3695,1489,725
131,1188,1980,673
153,2099,929,328
107,3787,1695,727
165,3339,1695,909
...,...,...,...
46,273,463,903
102,1821,1845,553
117,1292,1392,135
170,1953,1278,560


In [5]:
selected_data = data[:100]
other_nodes = data[100:]
# selected_data = selected_data.reset_index()
selected_data


,0,1,2
191,3695,1489,725
131,1188,1980,673
153,2099,929,328
107,3787,1695,727
165,3339,1695,909
...,...,...,...
100,3549,733,919
50,1812,325,328
5,3121,1452,558
10,778,785,837


In [6]:
other_nodes

,0,1,2
92,3637,451,454
22,1187,1284,169
69,1549,241,553
4,2125,360,72
148,3889,482,871
...,...,...,...
46,273,463,903
102,1821,1845,553
117,1292,1392,135
170,1953,1278,560


In [7]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)


    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm+temp+temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    
    if show: 
        df_dist = pd.DataFrame(dm)
        display(df_dist)
    return dm

In [8]:
dm = generate_dm(tsp_c)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,inf,1690.0,1022.0,997.0,672.0,2250.0,2008.0,1903.0,2522.0,1721.0,...,2292.0,2845.0,1067.0,738.0,1391.0,1097.0,2232.0,3084.0,2821.0,1261.0
1,1690.0,inf,1420.0,2642.0,1103.0,1730.0,2081.0,3212.0,1476.0,1255.0,...,1986.0,1996.0,1002.0,2383.0,2161.0,2485.0,1572.0,2372.0,1404.0,664.0
2,1022.0,1420.0,inf,1976.0,436.0,2087.0,2057.0,2599.0,2263.0,1479.0,...,2207.0,2627.0,821.0,1705.0,1712.0,1833.0,2028.0,2909.0,2532.0,1056.0
3,997.0,2642.0,1976.0,inf,1627.0,3131.0,2772.0,2205.0,3450.0,2652.0,...,3128.0,3750.0,2020.0,1115.0,2059.0,1438.0,3135.0,3960.0,3776.0,2203.0
4,672.0,1103.0,436.0,1627.0,inf,1792.0,1762.0,2277.0,1965.0,1183.0,...,1913.0,2331.0,493.0,1358.0,1405.0,1506.0,1734.0,2614.0,2222.0,725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1097.0,2485.0,1833.0,1438.0,1506.0,2434.0,1798.0,903.0,2900.0,2220.0,...,2295.0,3092.0,1924.0,1301.0,1069.0,inf,2513.0,3196.0,3487.0,2159.0
196,2232.0,1572.0,2028.0,3135.0,1734.0,768.0,1280.0,3052.0,985.0,1351.0,...,1044.0,1205.0,1821.0,2847.0,1905.0,2513.0,inf,1475.0,1875.0,1783.0
197,3084.0,2372.0,2909.0,3960.0,2614.0,1391.0,1570.0,3614.0,1154.0,2236.0,...,1287.0,1119.0,2706.0,3675.0,2544.0,3196.0,1475.0,inf,2291.0,2646.0
198,2821.0,1404.0,2532.0,3776.0,2222.0,2045.0,2565.0,4146.0,1275.0,2066.0,...,2306.0,1913.0,2144.0,3507.0,3011.0,3487.0,1875.0,2291.0,inf,1827.0


In [9]:
def get_new_node(id_node_before:int, id_node_after:int, cost, other_ids):
#     node_before = 2
#     node_after = 4
#     print(id_node_before,id_node_after)
    node = None
    for new_node in other_ids:
#         print(new_node)
        new_cost = dm[new_node][id_node_before] + dm[new_node][id_node_after]
        if new_cost < cost:
            cost = new_cost
            node = new_node
#     print(node)
    return node
    

In [10]:

def c(path):
    total = 0
    for idx, node in enumerate(path):
        total += dm[node][path[(idx+1)%100]]
    return total


In [11]:
# original_path = selected_data
def execute(selected_data,other_nodes,dm):
    path = selected_data.index.to_list()
    other_nodes_ids = other_nodes.index.to_list()
#     print(other_nodes_ids)
    solution = True
    while solution:
        new_node = None 
        solution = False
#         print(path)
        for idx, node_id in enumerate(path):
            try:
                node_cost = dm[node_id][path[idx-1]] + dm[node_id][path[idx+1]]
                new_node = get_new_node(path[idx-1],path[idx+1],node_cost,other_nodes_ids)
            except IndexError:
                node_cost = dm[node_id][path[idx-1]] + dm[node_id][path[0]]
                new_node = get_new_node(path[idx-1],path[0],node_cost,other_nodes_ids)
#             print(new_node)
            if new_node:
                other_nodes_ids.append(node_id)
                other_nodes_ids.remove(new_node)
                path[idx] = new_node
                solution = True
        
#         print(path)
        print(c(path))
    return path
        
#     print(new_node)


In [18]:
data = tsp_c.sample(frac=1)
selected_data = data[:100]
other_nodes = data[100:]
t1 = time.time()
path = execute(selected_data,other_nodes,dm)
print(time.time()-t1)

122751.0
99471.0
91950.0
89395.0
87921.0
86934.0
85639.0
85278.0
85242.0
85242.0
0.06571292877197266


In [19]:
def get_new_node_and_cost(id_node_before:int, id_node_after:int, cost, other_ids):
#     node_before = 2
#     node_after = 4
#     print(id_node_before,id_node_after)
    original_cost = cost
    node = None
    for new_node in other_ids:
#         print(new_node)
        new_cost = dm[new_node][id_node_before] + dm[new_node][id_node_after]
        if new_cost < cost:
            cost = new_cost
            node = new_node
#     print(node)
    return node, original_cost-cost

In [38]:
# original_path = selected_data
def execute_2(selected_data,other_nodes,dm):
    path = selected_data.index.to_list()
    other_nodes_ids = other_nodes.index.to_list()
#     print(other_nodes_ids)
    solution = True
#     print('origi',c(path))
    while solution:
        possible_solutions = []
        new_node = None 
        solution = False
        for idx, node_id in enumerate(path):
            try:
                node_cost = dm[node_id][path[idx-1]] + dm[node_id][path[idx+1]]
                new_node,new_cost = get_new_node_and_cost(path[idx-1],path[idx+1],node_cost,other_nodes_ids)
            except IndexError:
                node_cost = dm[node_id][path[idx-1]] + dm[node_id][path[0]]
                new_node,new_cost = get_new_node_and_cost(path[idx-1],path[0],node_cost,other_nodes_ids)
            if new_node:
                possible_solutions.append((node_id,idx,new_node,new_cost))
#                 other_nodes_ids.append(node_id)
#                 other_nodes_ids.remove(new_node)
#                 path[idx] = new_node
#                 solution = True
#         print('posss_col',possible_solutions)
        try:
            best_node = max(possible_solutions,key=lambda x: x[-1])
#             print('best_node',best_node)

#             break
            other_nodes_ids.append(best_node[0])
            other_nodes_ids.remove(best_node[2])
            path[best_node[1]] = best_node[2]
            solution = True
        except ValueError:
            print('dddd')
            break
#         print('best_node',best_node)

#         print(best_node)
        print(c(path))
    return path
        
#     print(new_node)

In [45]:
data = tsp_c.sample(frac=1)
selected_data = data[:100]
other_nodes = data[100:]
t1 = time.time()
path = execute_2(selected_data,other_nodes,dm)
print(time.time()-t1)

205853.0
200485.0
195564.0
190791.0
186314.0
181936.0
177668.0
173415.0
169179.0
165026.0
161050.0
157725.0
154598.0
151481.0
148514.0
145588.0
142922.0
140393.0
137890.0
135510.0
133435.0
131371.0
129328.0
127462.0
125649.0
123861.0
122094.0
120361.0
118692.0
117148.0
115620.0
114099.0
112682.0
111277.0
109927.0
108579.0
107498.0
106435.0
105460.0
104488.0
103572.0
102534.0
101593.0
100530.0
99639.0
98226.0
97200.0
96479.0
94211.0
93420.0
92709.0
92032.0
91384.0
90777.0
90186.0
89633.0
89133.0
88000.0
87555.0
87128.0
86692.0
86267.0
85860.0
85480.0
85113.0
84759.0
84256.0
83905.0
83554.0
83234.0
82859.0
82546.0
82237.0
81977.0
81768.0
81390.0
81198.0
80874.0
80591.0
80404.0
80231.0
80062.0
79912.0
79769.0
79489.0
78926.0
78797.0
78609.0
78485.0
78381.0
78313.0
78246.0
78195.0
78161.0
77926.0
77907.0
77869.0
77759.0
77752.0
77747.0
77642.0
dddd
0.4357607364654541


In [29]:
len(path) == len(set(path))

True